### Hypothesis testing: Lab 7

Elements of Data Science

In [ ]:
import numpy as np
from datascience import *

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Fix for datascience plots
import collections as collections
import collections.abc as abc
collections.Iterable = abc.Iterable

## Hypothesis Testing

**Question 7: Complete the simulation below, which samples num_changes increases/decreases at random many times and forms an empirical distribution of your test statistic under the null hypothesis.** 

Your job is to:
fill in the function simulate_under_null, which simulates a single sample under the null hypothesis, and
fill in its argument when it's called below.
**As a hint, num_changes should be approximately the number of countries times the number of time comparisons (you can find the number of year comparisons by using diff_n().**


## How to approach the problem

### Step 1: Make sure you understand the part of the function that was given to you.

In [ ]:
uniform = Table().with_columns(
    "Change", make_array('Increase', 'Decrease'),
    "Chance", make_array(0.5,        0.5))

uniform

### Step 2: Try the sample the sample_from_distribution method of datatables.

In [ ]:
sample = uniform.sample_from_distribution('Chance', 20)
sample

### Step 3: Add this to your function
For the each sample created by the simulation we have to simulate having the same amount of data as in our observation.

In [ ]:
def simulate_under_null(num_chances_to_change):
    """Simulates some number changing several times, with an equal
    chance to increase or decrease.  Returns the value of your
    test statistic for these simulated changes.
    
    num_chances_to_change is the number of times the number changes.
    """
    uniform = Table().with_columns(
        "Change", make_array('Increase', 'Decrease'),
        "Chance", make_array(0.5,        0.5))
    sample = uniform.sample_from_distribution('Chance', num_chances_to_change)
    return sample

In [ ]:
sample = simulate_under_null(10)
sample

## Step 4: We want to return the net positive increases

In [ ]:
sample.column('Chance sample')

In [ ]:
net = sample.column('Chance sample')[0] - sample.column('Chance sample')[1]
net

## Step 5: Add this to you function, too.

In [ ]:
def simulate_under_null(num_chances_to_change):
    """Simulates some number changing several times, with an equal
    chance to increase or decrease.  Returns the value of your
    test statistic for these simulated changes.
    
    num_chances_to_change is the number of times the number changes.
    """
    uniform = Table().with_columns(
        "Change", make_array('Increase', 'Decrease'),
        "Chance", make_array(0.5,        0.5))
    sample = uniform.sample_from_distribution('Chance', num_chances_to_change)
    net = sample.column('Chance sample')[0] - sample.column('Chance sample')[1]
    return net

In [ ]:
sample = simulate_under_null(10)
sample

## Step 6: Look at the second function

The function is nearly complete. It just wants your to figure out the correct value for num_changes. **Look back at the hint!**
"As a hint, num_changes should be approximately the number of countries times the number of time comparisons (you can find the number of year comparisons by using diff_n()."

**Okay, this is not a great hint.**

Let't look at a test table to figure this out.

In [ ]:
# Example Table
example = Table().with_columns(
    "year",
    make_array(2000, 2001, 2002, 2003, 2000, 2001, 2002, 2003),
    "country",
    make_array("France", "France", "France", "France", "England", "England", "England", "England"),
    "avg T",
    make_array(23, 24, 24, 25, 19, 21, 23, 23)
)
example

How many comparisons do we have? If the time step is 1 year, there will be three for France and
three for England. The number of comparisons is one less then the number of rows for each country. 

The number of comparisons is roughly the number of countries times the number of years, and as the size of the table increases this become a very close approximation. Using the diff_n function only changes the answer a little bit.

In [ ]:
def diff_n(values, n):
    return np.array(values)[n:] - np.array(values)[:-n]

diff_n(make_array(1, 10, 100, 1000, 10000), 2)

Notice that the number of changes if the step size is 2 is just two less than the number of values in the array, so using the number of values in the array will be close.

So how do we find the number of countries and the number of years of data?

In [ ]:
np.unique(example.column('country'))

In [ ]:
num_countries = len(np.unique(example.column('country')))
num_years = len(np.unique(example.column('year')))
num_changes = num_countries * num_years
num_changes

So, for our table `temps`

```python
num_countries = len(np.unique(temps.column('country')))
num_years = len(np.unique(temps.column('year')))
num_changes = num_countries * num_years
num_changes
```

Which we can use in the function below.

Or we can just use the number of rows in the table.

In [ ]:
example.num_rows

In [ ]:
def empirical_distribution(tbl):
    num_changes = ...
    samples = make_array()
    for i in np.arange(10000):
        samples = np.append(samples, simulate_under_null(...)) 
    Table().with_column('Test statistic under null', samples).hist(bins=np.arange(0, max(samples) + max(samples) * 0.1, 2))
    return samples

This function is ready for you to pass in the table: changes_by_country after you fix num_changes.

## HINTS: Mount St. Helens Mini-project

In [ ]:
# Plot description dataset
datafile = "../../Mini Project II/data/MSH_PLOT_DESCRIPTORS2.csv"
MSH_PLOT = Table.read_table(datafile)
MSH_PLOT.where('PLOT_CODE','ABPL10')

In [ ]:
np.unique(MSH_PLOT['PLOT_CODE'])

In [ ]:
MSH_PLOT = MSH_PLOT.with_columns('NLONG', -1*(MSH_PLOT.column('LONG')))
MSH_map = MSH_PLOT.select('LAT', 'NLONG', 'PLOT_CODE').relabel('PLOT_CODE', 'labels')
Circle.map_table(MSH_map, color='blue',area=200)

In [ ]:
datafile = "../../Mini Project II/data/MSH_STRUCTURE_PLOT_YEAR.csv"
MSH_YEAR = Table.read_table(datafile)
MSH_YEAR.show(4)

In [ ]:
np.unique(MSH_YEAR['PLOT_NAME'])

## Comparing plots

In [ ]:
MSH_YEAR.where('PLOT_NAME','ABPL').scatter("YEAR","COVER_%")

In [ ]:
MSH_YEAR.where('PLOT_NAME','STRD').scatter("YEAR","COVER_%")

### Maybe compare average COVER_% for a given year?

In [ ]:
abpl_2000 = MSH_YEAR.where("YEAR", are.equal_to(2000)).where("PLOT_NAME", are.equal_to("ABPL"))
abpl_2000.show(3)

In [ ]:
strd_2000 = MSH_YEAR.where("YEAR", are.equal_to(2000)).where("PLOT_NAME", are.equal_to("STRD"))
strd_2000.show(3)

In [ ]:
two_plots = abpl_2000.append(strd_2000)
two_plots.hist("RICHNESS", group="PLOT_NAME", bins = 20)

In [ ]:
two_plots.group("PLOT_NAME", np.mean)

### Is the difference in average richness between the two plot you chose statistically significant?

# Trends over time

In [ ]:
MSH_YEAR.where('PLOT_NAME','ABPL').group("YEAR",np.mean)

In [ ]:
MSH_YEAR.where('PLOT_NAME','ABPL').group("YEAR",np.mean).scatter("YEAR", "COVER_% mean", color='green')
plt.xlim(1995,2010)

## Is this trend random or non-random?  (Same idea as Lab07)